<a href="https://colab.research.google.com/github/emilie-xu/apam4903/blob/main/Song_Regressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://www1.columbia.edu/~chw2/Courses/APMA4903/2023/spotify-2023.csv', encoding='ISO-8859-1')

In [2]:
df.columns.values.tolist()

['track_name',
 'artist(s)_name',
 'artist_count',
 'released_year',
 'released_month',
 'released_day',
 'in_spotify_playlists',
 'in_spotify_charts',
 'streams',
 'in_apple_playlists',
 'in_apple_charts',
 'in_deezer_playlists',
 'in_deezer_charts',
 'in_shazam_charts',
 'bpm',
 'key',
 'mode',
 'danceability_%',
 'valence_%',
 'energy_%',
 'acousticness_%',
 'instrumentalness_%',
 'liveness_%',
 'speechiness_%']

In [3]:
df['artist(s)_name'].value_counts()

Taylor Swift                 34
The Weeknd                   22
Bad Bunny                    19
SZA                          19
Harry Styles                 17
                             ..
Karol G, Ovy On The Drums     1
Coolio, L.V.                  1
Kordhell                      1
Kenia OS                      1
Feid, Sech, Jhayco            1
Name: artist(s)_name, Length: 645, dtype: int64

##Preprocessing
Create function that labels song likeability label by specific individual given an array of liked songs and an array of disliked songs

In [4]:
#preprocessing (not needed for logistic regression)

#turn mode into binary
#df['isMajor'] = df['mode']=='Major'

#get relevant numerical features
numerical_labels = df._get_numeric_data().columns.values.tolist()
numerical_labels
num_features = numerical_labels[9:]
print(num_features)

ohe_features = []

['bpm', 'danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']


In [5]:
def extract_title_artist(song_arr):
  titles = []
  artists = []
  for info in song_arr:
    t, a = info.rsplit(' - ', 1)
    titles.append(t)
    artists.append(a)
  return titles, artists

def generate_labels(df, liked, disliked):
  liked_titles, liked_artists = extract_title_artist(liked)
  disliked_titles, disliked_artists = extract_title_artist(disliked)
  all_titles = liked_titles + disliked_titles
  training = df[df['track_name'].isin(all_titles)]
  testing = df[~df['track_name'].isin(all_titles)]
  training['liked_label']=np.where(training['track_name'].isin(liked_titles), 1, -1)
  #training.loc[training['track_name'].isin(disliked_titles), 'liked_label'] = -1
  return training, testing

In [6]:
emily_likes = ["Iris - The Goo Goo Dolls",
               "Labyrinth - Taylor Swift",
               "Mr. Brightside - The Killer",
               "Do I Wanna Know? - Arctic Monkeys",
               "Everybody Wants To Rule The World - Tears For Fears"]
emily_dislikes = ["Dance Monkey - Tones and I",
                  "If We Ever Broke Up - Mae Stephens",
                  "Light Switch - Charlie Puth",
                  "Bad Habits - Ed Sheeran",
                  "Ghost - Justin Bieber"]
liz_likes = ["Take Me To Church - Hozier",
             "august - Taylor Swift",
             "Matilda - Harry Styles",
             "Easy On Me - Adele",
             "Let Me Down Slowly - Alec Benjamin"]
liz_dislikes = ["golden hour - JVKE",
                "Unholy (feat. Kim Petras) - Sam Smith, Kim Petras",
                "Unstoppable - Sia",
                "Bad Habits - Ed Sheeran",
                "Made You Look - Meghan Trainor"]

## Logistic Regression

In [7]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
import numpy as np

feature_names = num_features + ohe_features

def SongLogisticRegression(likes, dislikes):
  train, test = generate_labels(df, likes, dislikes)
  dev_y = train.loc[:, ['liked_label']]
  dev_X = train.loc[:, train.columns != 'liked_label']
  test_X = test.loc[:, test.columns != 'liked_label']

  #dev_X, test_X, dev_y, test_y = train_test_split(df, target, test_size=0.2)
  preprocess = make_column_transformer((StandardScaler(), num_features))
  logisticRegr = LogisticRegression()
  pipe = make_pipeline(preprocess, logisticRegr)
  #scores = cross_val_score(pipe, dev_X, dev_y, cv=10, error_score="raise")

  pipe.fit(dev_X, dev_y)
  pipe.predict(test_X)
  test_X['pred'] = pipe.predict(test_X)
  return test_X

In [8]:
emily_logistic = SongLogisticRegression(emily_likes, emily_dislikes)
emily_logistic.sort_values('pred', ascending=False).head(15)

<ipython-input-5-e9a4bec74d35>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training['liked_label']=np.where(training['track_name'].isin(liked_titles), 1, -1)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,pred
952,Alone,Burna Boy,1,2022,11,4,782,2,96007391,27,...,E,Minor,61,32,67,15,0,11,5,1
399,TV,Billie Eilish,1,2022,7,21,3009,2,338564981,68,...,E,Minor,41,9,25,84,1,14,4,1
731,Fuera del mercado,Danny Ocean,1,2022,2,17,2499,21,421365166,68,...,G#,Major,45,27,67,32,0,13,14,1
732,X ï¿½ï¿½LTIMA,"Daddy Yankee, Bad Bunny",2,2022,3,25,2697,1,349746291,55,...,G,Major,81,59,83,9,0,11,5,1
395,Space Song,Beach House,1,2015,1,1,17852,4,789753877,69,...,NaN,Minor,51,62,79,22,13,14,3,1
394,Escapism. - Sped Up,"RAYE, 070 Shake",2,2022,11,25,1368,0,184308753,12,...,F#,Minor,44,38,77,9,0,9,20,1
734,In My Head,Lil Tjay,1,2022,4,1,1185,0,190981339,21,...,C#,Major,68,41,55,16,0,10,12,1
735,Wait a Minute!,Willow,1,2015,1,11,11985,0,924193303,79,...,D#,Minor,76,63,71,3,0,10,3,1
388,STAR WALKIN' (League of Legends Worlds Anthem),Lil Nas X,1,2022,9,22,2616,0,332506354,113,...,D,Minor,64,31,72,15,0,9,5,1
387,Lift Me Up - From Black Panther: Wakanda Forev...,Rihanna,1,2022,10,28,3311,0,297328960,129,...,A,Major,25,17,30,90,0,13,3,1


In [9]:
liz_logistic = SongLogisticRegression(liz_likes, liz_dislikes)
liz_logistic.sort_values('pred', ascending=False).head(15)

<ipython-input-5-e9a4bec74d35>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training['liked_label']=np.where(training['track_name'].isin(liked_titles), 1, -1)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,pred
270,All Of The Girls You Loved Before,Taylor Swift,1,2019,8,23,1282,6,185240616,26,...,D,Major,72,40,47,71,0,13,4,1
725,Closer,"The Chainsmokers, Halsey",2,2016,5,31,28032,0,2591224264,315,...,G#,Major,75,64,52,41,0,11,3,1
710,Chale,Eden Muï¿½ï,1,2022,2,18,695,11,299648208,16,...,G#,Major,55,86,44,40,0,7,4,1
711,DARARI,Treasure,1,2022,2,15,328,0,182978249,10,...,NaN,Major,72,55,64,49,0,7,4,1
715,this is what falling in love feels like,JVKE,1,2021,9,3,2005,0,346127840,16,...,B,Major,42,33,44,62,0,8,6,1
717,Bohemian Rhapsody - Remastered 2011,Queen,1,1975,10,31,40112,3,2197010679,321,...,NaN,Minor,41,23,40,27,0,30,5,1
718,Hope,XXXTENTACION,1,2018,3,16,3659,0,1200808494,11,...,A,Minor,59,23,46,66,0,15,6,1
288,I Was Never There,"The Weeknd, Gesaffelstein",2,2018,3,29,4188,15,705469769,30,...,A#,Major,32,17,74,14,0,17,3,1
727,Somebody That I Used To Know,"Gotye, Kimbra",2,2011,1,1,42798,0,1457139296,217,...,NaN,Major,86,75,52,54,0,10,4,1
750,Falling,Harry Styles,1,2019,12,13,7556,0,1023187129,124,...,E,Major,57,6,27,84,0,9,3,1


## Support Vector Machines (SVMs)

In [10]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import LinearSVC
import numpy as np

feature_names = num_features + ohe_features

#print(feature_names)
def SongsSVM(likes, dislikes):
  train, test = generate_labels(df, likes, dislikes)
  dev_y = train.loc[:, ['liked_label']]
  dev_X = train.loc[:, train.columns != 'liked_label']
  test_X = test.loc[:, test.columns != 'liked_label']

  preprocess = make_column_transformer((StandardScaler(), num_features))
  svmRegr = LinearSVC(C=0.1, penalty='l2', loss='squared_hinge', dual=True)
  pipe = make_pipeline(preprocess, svmRegr)
  #scores = cross_val_score(pipe, dev_X, dev_y, cv=10, error_score="raise")

  pipe.fit(dev_X, dev_y)
  pipe.predict(test_X)
  test_X['pred'] = pipe.predict(test_X)
  return test_X

In [11]:
emily_svm = SongsSVM(emily_likes, emily_dislikes)
emily_svm.sort_values('pred', ascending=False).head(15)

<ipython-input-5-e9a4bec74d35>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training['liked_label']=np.where(training['track_name'].isin(liked_titles), 1, -1)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,pred
952,Alone,Burna Boy,1,2022,11,4,782,2,96007391,27,...,E,Minor,61,32,67,15,0,11,5,1
243,Unstoppable,Sia,1,2016,1,21,7681,13,939844851,119,...,A,Major,47,27,78,11,0,10,8,1
734,In My Head,Lil Tjay,1,2022,4,1,1185,0,190981339,21,...,C#,Major,68,41,55,16,0,10,12,1
388,STAR WALKIN' (League of Legends Worlds Anthem),Lil Nas X,1,2022,9,22,2616,0,332506354,113,...,D,Minor,64,31,72,15,0,9,5,1
387,Lift Me Up - From Black Panther: Wakanda Forev...,Rihanna,1,2022,10,28,3311,0,297328960,129,...,A,Major,25,17,30,90,0,13,3,1
735,Wait a Minute!,Willow,1,2015,1,11,11985,0,924193303,79,...,D#,Minor,76,63,71,3,0,10,3,1
738,Chance,Paulo Londra,1,2022,4,6,225,0,89566512,11,...,C#,Minor,72,22,46,24,0,9,6,1
382,TRUSTFALL,P!nk,1,2023,1,27,2098,16,134255790,88,...,G#,Major,64,25,89,0,0,15,9,1
739,Cool for the Summer,Demi Lovato,1,2015,1,1,9243,0,677389855,155,...,F,Minor,59,30,62,1,0,8,4,1
379,Devil Donï¿½ï¿½ï¿½,Morgan Wallen,1,2023,3,3,166,0,32526947,2,...,NaN,Major,53,32,66,38,0,9,3,1


In [12]:
liz_svm = SongsSVM(liz_likes, liz_dislikes)
liz_svm.sort_values('pred', ascending=False).head(15)

<ipython-input-5-e9a4bec74d35>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training['liked_label']=np.where(training['track_name'].isin(liked_titles), 1, -1)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,pred
270,All Of The Girls You Loved Before,Taylor Swift,1,2019,8,23,1282,6,185240616,26,...,D,Major,72,40,47,71,0,13,4,1
288,I Was Never There,"The Weeknd, Gesaffelstein",2,2018,3,29,4188,15,705469769,30,...,A#,Major,32,17,74,14,0,17,3,1
704,"I'm Tired - From ""Euphoria"" An Original HBO Se...",Labrinth,1,2022,2,4,1888,0,121913181,26,...,NaN,Minor,28,26,20,19,0,30,3,1
710,Chale,Eden Muï¿½ï,1,2022,2,18,695,11,299648208,16,...,G#,Major,55,86,44,40,0,7,4,1
711,DARARI,Treasure,1,2022,2,15,328,0,182978249,10,...,NaN,Major,72,55,64,49,0,7,4,1
715,this is what falling in love feels like,JVKE,1,2021,9,3,2005,0,346127840,16,...,B,Major,42,33,44,62,0,8,6,1
717,Bohemian Rhapsody - Remastered 2011,Queen,1,1975,10,31,40112,3,2197010679,321,...,NaN,Minor,41,23,40,27,0,30,5,1
718,Hope,XXXTENTACION,1,2018,3,16,3659,0,1200808494,11,...,A,Minor,59,23,46,66,0,15,6,1
725,Closer,"The Chainsmokers, Halsey",2,2016,5,31,28032,0,2591224264,315,...,G#,Major,75,64,52,41,0,11,3,1
750,Falling,Harry Styles,1,2019,12,13,7556,0,1023187129,124,...,E,Major,57,6,27,84,0,9,3,1


## Boosting

In [13]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
import numpy as np

feature_names = num_features + ohe_features

def SongsBoosting(likes, dislikes):
  train, test = generate_labels(df, likes, dislikes)
  dev_y = train.loc[:, ['liked_label']]
  dev_X = train.loc[:, train.columns != 'liked_label']
  test_X = test.loc[:, test.columns != 'liked_label']

  preprocess = make_column_transformer((StandardScaler(), num_features))
  boostRegr = AdaBoostRegressor(DecisionTreeRegressor(), random_state=42)
  pipe = make_pipeline(preprocess, GridSearchCV(boostRegr, param_grid=[{'n_estimators': [50,100,200], 'learning_rate':np.logspace(-3,0,4)}]))
  #scores = cross_val_score(pipe, dev_X, dev_y, cv=10, error_score="raise")

  pipe.fit(dev_X, dev_y)
  pipe.predict(test_X)
  test_X['pred'] = pipe.predict(test_X)
  return test_X

In [14]:
emily_boosting = SongsBoosting(emily_likes, emily_dislikes)
emily_boosting.sort_values('pred', ascending=False).head(15)

<ipython-input-5-e9a4bec74d35>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training['liked_label']=np.where(training['track_name'].isin(liked_titles), 1, -1)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.p

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,pred
952,Alone,Burna Boy,1,2022,11,4,782,2,96007391,27,...,E,Minor,61,32,67,15,0,11,5,1.0
379,Devil Donï¿½ï¿½ï¿½,Morgan Wallen,1,2023,3,3,166,0,32526947,2,...,NaN,Major,53,32,66,38,0,9,3,1.0
726,O.O,NMIXX,1,2022,2,22,290,0,135444283,9,...,B,Minor,39,28,77,4,0,6,29,1.0
395,Space Song,Beach House,1,2015,1,1,17852,4,789753877,69,...,NaN,Minor,51,62,79,22,13,14,3,1.0
394,Escapism. - Sped Up,"RAYE, 070 Shake",2,2022,11,25,1368,0,184308753,12,...,F#,Minor,44,38,77,9,0,9,20,1.0
392,Calm Down,Rï¿½ï,1,2022,2,9,4013,10,445763624,107,...,B,Major,81,82,78,38,0,12,4,1.0
729,First Class,Jack Harlow,1,2022,4,8,8737,0,694525298,163,...,G#,Major,91,32,56,3,0,11,10,1.0
730,Plan A,Paulo Londra,1,2022,3,23,1105,0,240661097,32,...,NaN,Major,58,56,83,5,0,7,4,1.0
389,"Sex, Drugs, Etc.",Beach Weather,1,2016,11,4,3006,16,480507035,47,...,E,Minor,57,47,84,1,1,52,4,1.0
731,Fuera del mercado,Danny Ocean,1,2022,2,17,2499,21,421365166,68,...,G#,Major,45,27,67,32,0,13,14,1.0


In [15]:
liz_boosting = SongsBoosting(liz_likes, liz_dislikes)
liz_boosting.sort_values('pred', ascending=False).head(15)

<ipython-input-5-e9a4bec74d35>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training['liked_label']=np.where(training['track_name'].isin(liked_titles), 1, -1)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.p

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,pred
480,I Hate U,SZA,1,2021,12,3,4094,0,356709897,66,...,G,Minor,54,41,39,51,0,11,16,1.0
825,Flowers,Lauren Spencer Smith,1,2021,8,9,801,0,184826429,42,...,D#,Major,70,31,44,84,0,9,39,1.0
822,San Lucas,Kevin Kaarl,1,2019,11,7,407,1,244891912,5,...,G,Major,58,27,36,86,0,9,3,1.0
819,With you,"HA SUNG WOON, Jimin",2,2022,4,24,343,2,240580042,4,...,D#,Major,53,14,43,64,0,13,3,1.0
613,Talking To The Moon,Bruno Mars,1,2010,10,4,7109,2,1062956628,5,...,C#,Minor,52,7,61,51,0,11,3,1.0
614,you broke me first,Tate McRae,1,2020,4,17,6638,0,1180094974,167,...,E,Major,64,8,37,79,0,9,5,1.0
166,Every Breath You Take - Remastered 2003,The Police,1,1983,1,6,22439,19,1593270737,211,...,C#,Major,82,73,45,54,0,7,3,1.0
169,When I Was Your Man,Bruno Mars,1,2012,12,5,2420,11,1661187319,0,...,NaN,Major,60,43,27,94,0,14,4,1.0
812,I Tried to Tell Y'all,"Ugly Dray, Tesla Jnr",2,2022,5,6,1657,0,121077868,15,...,B,Major,71,37,52,62,0,11,36,1.0
618,Cï¿½ï¿½,Rauw Alejandro,1,2021,6,25,2636,3,465959382,36,...,A,Minor,77,26,63,58,0,11,5,1.0
